In [ ]:
!pip install pandas beautifulsoup4 lxml

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import zipfile
import os

def kmz_to_csv(kmz_file_path, csv_output_path):
    # 1. Open the KMZ file as a zip archive
    with zipfile.ZipFile(kmz_file_path, 'r') as kmz:
        # 2. Find the KML file inside the KMZ (usually named doc.kml)
        kml_filename = [f for f in kmz.namelist() if f.endswith('.kml')][0]
        
        # 3. Read the KML content
        with kmz.open(kml_filename, 'r') as kml_file:
            kml_content = kml_file.read()

    # 4. Parse the KML using BeautifulSoup
    soup = BeautifulSoup(kml_content, 'xml')
    
    # List to store extracted data
    data = []

    # 5. Iterate through each 'Placemark' (points of interest)
    for placemark in soup.find_all('Placemark'):
        name = placemark.find('name').text if placemark.find('name') else 'N/A'
        
        # Extract coordinates (Format in KML is usually: lon,lat,alt)
        coords_tag = placemark.find('coordinates')
        if coords_tag:
            coords_raw = coords_tag.text.strip()
            # Split coordinates (handling standard KML format)
            try:
                parts = coords_raw.split(',')
                longitude = parts[0]
                latitude = parts[1]
            except IndexError:
                longitude, latitude = None, None
        else:
            longitude, latitude = None, None

        # Append to data list
        data.append({        
            'x': longitude,
            'y': latitude,
            'Id': name,
        })

    # 6. Convert to DataFrame and Save to CSV
    df = pd.DataFrame(data)
    df.to_csv(csv_output_path, index=False)
    
    print(f"Success! Converted '{kmz_file_path}' to '{csv_output_path}'")
    return df

In [4]:
# Replace with your KMZ file path
input_kmz = './Flood level reports/รายงานการคาดการณ์ความลึกน้ำท่วมจากแบบจำลอง วันที่ 26 พฤศจิกายน 2568 เวลา 07.20 น StriX-4/SX4_20251126_0700.kmz'
output_csv = './flood_level_csv/' + os.path.basename(input_kmz).replace('.kmz', '.csv')
#output_csv = os.path.basename(input_kmz).replace('.kmz', '.csv')

if os.path.exists(input_kmz):
    df_result = kmz_to_csv(input_kmz, output_csv)
    display(df_result.head())
else:
    print(f"File {input_kmz} not found. Please upload the file to your notebook environment.")

Success! Converted './Flood level reports/รายงานการคาดการณ์ความลึกน้ำท่วมจากแบบจำลอง วันที่ 26 พฤศจิกายน 2568 เวลา 07.20 น StriX-4/SX4_20251126_0700.kmz' to './flood_level_csv/SX4_20251126_0700.csv'


,x,y,Id
0,100.2312499996667,7.294583333000004,1
1,100.2337499996667,7.294583333000004,2
2,100.2337499996667,7.294583333000004,3
3,100.2329166663333,7.29374999966667,4
4,100.2337499996667,7.29374999966667,5
